In [1]:
"""
Proyecto: Optimización Predictiva de Inventario
Autor: Willialms Aguilera
Fecha: 2026

Objetivo:
Este script conecta con SQL Server, extrae el dataset de predicción, 
crea variables predictivas, entrena un modelo de clasificación para 
anticipar quiebres de inventario y simula políticas de reposición.

Repositorio: Wild2Data/Optimizacion-predictiva-de-la-demanda...
"""

# Imports
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report


In [2]:
# --------------------------------------------------------------
# 🌐 Conexión a SQL Server
# --------------------------------------------------------------
# Creamos el engine con conexión de Windows Authentication

server = 'HANDROLLERS\\SQLEXPRESS'
database = 'AtlanticFlowSurfDW'

engine = create_engine(
    f"mssql+pyodbc://@{server}/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

# Query principal
query = "SELECT * FROM vw_dataset_prediccion"
df = pd.read_sql(query, engine)

# Convertimos fechas
df['fecha'] = pd.to_datetime(df['fecha'])

print("Datos cargados correctamente:", df.shape)


Datos cargados correctamente: (59184, 18)


In [3]:
# --------------------------------------------------------------
# 🔧 Feature Engineering
# --------------------------------------------------------------

# Orden temporal por ciudad y producto
df = df.sort_values(['ciudad', 'nombre_producto', 'fecha'])

# Lag de ventas para incorporar tendencia
df['ventas_lag_1'] = (
    df.groupby(['ciudad', 'nombre_producto'])['unidades_vendidas']
      .shift(1)
)

# Ratio ventas/stock (contexto de consumo vs disponibilidad)
df['ratio_ventas_stock'] = (
    df['unidades_vendidas'] / (df['stock_disponible'] + 1)
)

# Mapeo de lead time (logística)
lead_time_map = {
    'Urbana': 5,
    'Turística': 7,
    'Remota': 10
}

df['lead_time_dias'] = df['tipo_sucursal'].map(lead_time_map)

# Target binario para quiebre futuro (antes de reponer)
df['quiebre_futuro'] = np.where(
    df['quiebre_stock'].rolling(window=7, min_periods=1).max() == 1, 1, 0
)

# Eliminamos filas inválidas
df_model = df.dropna()

print("Feature engineering completado:", df_model.shape)


Feature engineering completado: (59130, 22)


In [4]:
# --------------------------------------------------------------
# 🤖 Entrenamiento de Modelo de Clasificación
# --------------------------------------------------------------

features = [
    'ventas_lag_1',
    'ratio_ventas_stock',
    'stock_disponible'
]

X = df_model[features]
y = df_model['quiebre_futuro']

tscv = TimeSeriesSplit(n_splits=5)
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=8,
    random_state=42,
    n_jobs=-1
)

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

print("Reporte de clasificación final:\n")
print(classification_report(y_test, y_pred))


Reporte de clasificación final:

              precision    recall  f1-score   support

           0       0.53      0.15      0.24      3767
           1       0.64      0.92      0.75      6088

    accuracy                           0.63      9855
   macro avg       0.59      0.54      0.49      9855
weighted avg       0.60      0.63      0.56      9855



In [5]:
# --------------------------------------------------------------
# 📦 Simulación de Decisiones de Reposición
# --------------------------------------------------------------

df_model['prob_quiebre'] = model.predict_proba(X)[:, 1]

def simular_reposicion(df, umbral, costo_quiebre=1000, costo_reposicion=150):
    df_sim = df.copy()
    df_sim['reponer'] = (df_sim['prob_quiebre'] >= umbral).astype(int)

    df_sim['resultado'] = 'ok'
    df_sim.loc[
        (df_sim['reponer'] == 1) & (df_sim['quiebre_futuro'] == 1),
        'resultado'
    ] = 'quiebre_evitar'

    df_sim.loc[
        (df_sim['reponer'] == 1) & (df_sim['quiebre_futuro'] == 0),
        'resultado'
    ] = 'falsa_alarma'

    df_sim.loc[
        (df_sim['reponer'] == 0) & (df_sim['quiebre_futuro'] == 1),
        'resultado'
    ] = 'quiebre_real'

    df_sim['costo'] = 0
    df_sim.loc[df_sim['resultado'] == 'quiebre_real', 'costo'] = costo_quiebre
    df_sim.loc[df_sim['resultado'] == 'falsa_alarma', 'costo'] = costo_reposicion

    resumen = df_sim['resultado'].value_counts(normalize=True)
    costo_total = df_sim['costo'].sum()
    return resumen, costo_total

umbrales = [0.3, 0.4, 0.5]
for u in umbrales:
    resumen, costo = simular_reposicion(df_model, umbral=u)
    print(f"\n🔎 Umbral = {u}")
    print(resumen)
    print(f"Costo total: ${costo:,.0f}")


C:\Users\wiloc\AppData\Local\Temp\ipykernel_37708\4007879452.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['prob_quiebre'] = model.predict_proba(X)[:, 1]



🔎 Umbral = 0.3
resultado
quiebre_evitar    0.575461
falsa_alarma      0.424438
ok                0.000101
Name: proportion, dtype: float64
Costo total: $3,764,550

🔎 Umbral = 0.4
resultado
quiebre_evitar    0.574903
falsa_alarma      0.421952
ok                0.002588
quiebre_real      0.000558
Name: proportion, dtype: float64
Costo total: $3,775,500

🔎 Umbral = 0.5
resultado
quiebre_evitar    0.345036
ok                0.261373
quiebre_real      0.230424
falsa_alarma      0.163166
Name: proportion, dtype: float64
Costo total: $15,072,200
